In [1]:
!pip install stellargraph

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import os
import csv
import pdb
import math
import time 
import pickle
import pickle
import random
import warnings
import datetime 
import collections
import numpy as np 
import pandas as pd  
import xgboost as xgb
import networkx as nx
from tqdm.notebook import tqdm
import matplotlib.pylab as plt 
from matplotlib import rcParams   
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import f1_score
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier
import time

# Algo import
from stellargraph.core import StellarGraph
from stellargraph.mapper import AdjacencyPowerGenerator
from stellargraph.layer import WatchYourStep
from stellargraph.losses import graph_log_likelihood
from stellargraph.utils import plot_history
import tensorflow
from tensorflow.keras import optimizers, Model, layers, regularizers
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [4]:
data_path = "gdrive/My Drive/Major Project"
data_path = "gdrive/My Drive/Major Project 8th sem/Major Project"

In [5]:
combined_tr = pd.read_csv(data_path+'/train_91.csv', index_col = [0])
print(combined_tr.shape)
combined_tr.head()

(902474, 3)


,src,dest,link
73132,35080,10701,0
404654,54809,75095,0
310475,67628,60404,0
301271,57253,462,0
332230,15167,47459,1


In [6]:
combined_te = pd.read_csv(data_path+'/test_91.csv', index_col = [0])
print(combined_te.shape)
combined_te.head()

(106363, 3)


,src,dest,link
487942,32198,62377,0
103900,6056,11650,1
78661,2040,22467,1
71074,143,0,1
382438,1609,5035,1


In [7]:
df_train = pd.read_csv(data_path+'/pos_train_91.csv', index_col = [0])
print(df_train.shape)
df_train.head()

(452474, 3)


,src,dest,link
496205,54500,16485,1
141147,3318,30547,1
483066,33514,4007,1
478351,28118,2641,1
505144,69985,61687,1


In [8]:
train_graph = nx.from_pandas_edgelist(df_train, source='src', target='dest', create_using=nx.MultiDiGraph())
G = StellarGraph.from_networkx(train_graph)
generator = AdjacencyPowerGenerator(G, num_powers = 3)

/usr/local/lib/python3.7/dist-packages/stellargraph/core/utils.py:134: RuntimeWarning: divide by zero encountered in float_power
  d = sp.diags(np.float_power(np.array(adj.sum(1)), -1).flatten(), 0)


In [9]:
wys = WatchYourStep(
    generator,
    num_walks=80,
    embedding_dimension=64,
    attention_regularizer=regularizers.l2(0.5),
)
x_in, x_out = wys.in_out_tensors()

In [10]:
model = Model(inputs = x_in, outputs = x_out)
model.compile(loss=graph_log_likelihood, optimizer=tf.keras.optimizers.Adam(1e-2))
es = EarlyStopping(
    monitor = 'loss',
    patience = 2,
    verbose = 1,
    restore_best_weights = True
)

In [11]:
epochs = 20
batch_size = 128
train_gen = generator.flow(batch_size=batch_size)
history = model.fit(
    train_gen, epochs=epochs, verbose=1, steps_per_epoch=int(len(G.nodes()) // batch_size), callbacks = [es]
)

Epoch 1/20


/usr/local/lib/python3.7/dist-packages/stellargraph/core/experimental.py:12: ExperimentalWarning: graph_log_likelihood is experimental: lack of unit tests (see: https://github.com/stellargraph/stellargraph/issues/804). It may be difficult to use and may have major changes at any time.
  # distributed under the License is distributed on an "AS IS" BASIS,


592/592 [==============================] - 614s 1s/step - loss: 6737449.0000
Epoch 2/20
592/592 [==============================] - 612s 1s/step - loss: 586039.5000
Epoch 3/20
592/592 [==============================] - 600s 1s/step - loss: 63679.0547
Epoch 4/20
592/592 [==============================] - 596s 1s/step - loss: 50496.3711
Epoch 5/20
592/592 [==============================] - 596s 1s/step - loss: 45573.3555
Epoch 6/20
592/592 [==============================] - 594s 1s/step - loss: 42503.9883
Epoch 7/20
592/592 [==============================] - 593s 1s/step - loss: 40553.5742
Epoch 8/20
592/592 [==============================] - 591s 998ms/step - loss: 39370.1953
Epoch 9/20
592/592 [==============================] - 593s 1s/step - loss: 38781.4844
Epoch 10/20
592/592 [==============================] - 594s 1s/step - loss: 39156.1992
Epoch 11/20
592/592 [==============================] - 593s 1s/step - loss: 40642.4180
Epoch 11: early stopping


In [12]:
model_file = open(data_path+"epinions_wys_model.h5", "wb")
pickle.dump(model, model_file)
model_file.close()

INFO:tensorflow:Assets written to: ram://bfb58094-2235-40b1-bda4-f1986ca77716/assets


In [13]:
embeddings = wys.embeddings()

In [14]:
emb_file = open(data_path+"epinions_wys_embeddings.pkl", "wb")
pickle.dump(embeddings, emb_file)
emb_file.close()

In [15]:
emb_file = open(data_path+"epinions_wys_embeddings.pkl", "rb")
embeddings = pickle.load(emb_file)
emb_file.close()
embeddings.shape

(75879, 64)

In [16]:
nodes = list(G.nodes())

In [17]:
emb_dict = {}
for i, node in enumerate(nodes):
  emb_dict[node] = np.array(embeddings[i].reshape(1,64))

In [24]:
# get training embeddings
train_emb = []
for i, row in tqdm(combined_tr.iterrows()):
  comb_emb = emb_dict[row['src']][0] * emb_dict[row['dest']][0]
  train_emb.append(comb_emb)
train_emb = np.array(train_emb)
print(train_emb.shape)
train_y = combined_tr['link']
print(len(train_y))

0it [00:00, ?it/s]

(902474, 64)
902474


In [25]:
# get test embeddings
test_emb = []
for i, row in tqdm(combined_te.iterrows()):
  comb_emb = emb_dict[row['src']][0] * emb_dict[row['dest']][0]
  test_emb.append(comb_emb)

test_emb = np.array(test_emb)
print(test_emb.shape)
test_y = combined_te['link'] 
print(len(test_y))

0it [00:00, ?it/s]

(106363, 64)
106363


In [26]:
pickle.dump((train_emb, train_y), open(data_path + 'epinions_wys_train91.pkl', 'wb'))
pickle.dump((test_emb, test_y), open(data_path + 'epinions_wys_test91.pkl', 'wb'))

In [27]:
train_emb, train_y = pickle.load(open(data_path + 'epinions_wys_train91.pkl', 'rb'))
test_emb, test_y = pickle.load(open(data_path + 'epinions_wys_test91.pkl', 'rb'))

In [28]:
# train_emb_sample, _, train_y_sample, _ = train_test_split(train_emb, train_y, stratify = train_y, test_size = 0.5, random_state = 42)

In [29]:
clf = XGBClassifier(max_depth=7, n_estimators=100, n_jobs=-1,
              random_state=25)
start = time.time()
clf.fit(train_emb, train_y)
print("Time taken for XGB fit = ", time.time() - start)
start = time.time()
train_sc = f1_score(train_y,clf.predict(train_emb))
print("Time taken for train data precition = ", time.time() - start)
start = time.time()
test_sc = f1_score(test_y,clf.predict(test_emb))
print("Time taken for test data precition = ", time.time() - start)
print('Train Score',train_sc,'test Score',test_sc)

Time taken for XGB fit =  869.468489408493
Time taken for train data precition =  4.341338872909546
Time taken for test data precition =  0.5011496543884277
Train Score 0.9068229592268027 test Score 0.9105722001703732


In [30]:
clf = RandomForestClassifier(max_depth=12, n_estimators=100, n_jobs=-1,
              random_state=25)
start = time.time()
clf.fit(train_emb, train_y)
print("Time taken for RandomForest fit = ", time.time() - start)
start = time.time()
train_sc = f1_score(train_y,clf.predict(train_emb))
print("Time taken for train data precition = ", time.time() - start)
start = time.time()
test_sc = f1_score(test_y,clf.predict(test_emb))
print("Time taken for test data precition = ", time.time() - start)
print('Train Score',train_sc,'test Score',test_sc)

Time taken for RandomForest fit =  660.378564119339
Time taken for train data precition =  7.880524396896362
Time taken for test data precition =  0.9537849426269531
Train Score 0.9096481635456583 test Score 0.9102567585925825


In [31]:
clf = LGBMClassifier(boosting_type='gbdt', num_leaves=31, max_depth=-1, learning_rate=0.1, n_estimators=100, subsample_for_bin=200000, 
                     objective=None, class_weight=None, min_split_gain=0.0, min_child_weight=0.001, min_child_samples=20, subsample=1.0, 
                     subsample_freq=0, colsample_bytree=1.0, reg_alpha=0.0, reg_lambda=0.0, random_state=None, n_jobs=- 1, importance_type='split')
start = time.time()
clf.fit(train_emb, train_y)
print("Time taken for LGBM fit = ", time.time() - start)
start = time.time()
train_sc = f1_score(train_y,clf.predict(train_emb))
print("Time taken for train data precition = ", time.time() - start)
start = time.time()
test_sc = f1_score(test_y,clf.predict(test_emb))
print("Time taken for test data precition = ", time.time() - start)
print('Train Score',train_sc,'test Score',test_sc)

Time taken for LGBM fit =  33.53799104690552
Time taken for train data precition =  4.7518470287323
Time taken for test data precition =  0.559330940246582
Train Score 0.9032055211958384 test Score 0.909011778146506
